In [1]:
import types
from functools import partial
import dis
import math
from ctypes import c_uint8
from functools import reduce
import re

In [2]:
def print_co(func):
    for attr in dir(func.__code__):
        if attr.startswith('co_'):
            print("\t%s = %s" % (attr, func.__code__.__getattribute__(attr)))

In [3]:
def copy_func(f, name=None):
    fn = types.FunctionType(f.__code__, f.__globals__, name or f.__name__,
                            f.__defaults__, f.__closure__)
    
    # In case f was given attrs (note this dict is a shallow copy):
    fn.__dict__.update(f.__dict__)
    
    return fn

In [4]:
types.FunctionType?

In [5]:
def replace(string, substitutions):
    substrings = sorted(substitutions, key=len, reverse=True)
    regex = re.compile(b'|'.join(map(re.escape, substrings)))
    return regex.sub(lambda match: substitutions[match.group(0)], string)

In [6]:
def tuple_remove_duplicate(tuple_):
    return tuple(sorted(set(tuple_), key=tuple_.index))

def replace_load_fast_by_load_const(bytecode, varname_index2const_index):
    varname_index2const_index = {b'|' + c_uint8(fast_index): b'd' + c_uint8(const_index) for fast_index, const_index in varname_index2const_index.items()}
    return replace(bytecode, varname_index2const_index)

def replace_fast_by_fast(bytecode, varname_index2varname_new_index):
    # STORE_FAST
    store_varname_index2varname_new_index = {b'}' + c_uint8(fast_index): b'}' + c_uint8(fast_new_index) for fast_index, fast_new_index in varname_index2varname_new_index.items()}
    bytecode = replace(bytecode, store_varname_index2varname_new_index)
    
    # LOAD_FAST
    load_varname_index2varname_new_index = {b'|' + c_uint8(fast_index): b'|' + c_uint8(fast_new_index) for fast_index, fast_new_index in varname_index2varname_new_index.items()}
    bytecode = replace(bytecode, load_varname_index2varname_new_index)
    
    return bytecode

In [8]:
def inlined_partial(func, name, **arg_name2value):
    for arg_name in arg_name2value:
        if arg_name not in func.__code__.co_varnames:
            raise KeyError(arg_name + " is not an argument of " + str(func))
            
    fcode = func.__code__
    new_consts = tuple_remove_duplicate(fcode.co_consts + tuple(arg_name2value.values()))
    varname_index2new_const_index = {fcode.co_varnames.index(arg_name): new_consts.index(value) for arg_name, value in arg_name2value.items()}
    
    new_varnames = tuple(set(fcode.co_varnames) - set(arg_name2value.keys()))
    varname_index2varname_new_index = {fcode.co_varnames.index(arg_name): new_varnames.index(arg_name) for arg_name in new_varnames}
    
    new_co_code = replace_load_fast_by_load_const(fcode.co_code, varname_index2new_const_index)
    new_co_code = replace_fast_by_fast(new_co_code, varname_index2varname_new_index)

    new_func = copy_func(func, name)
    
    nfcode = new_func.__code__
    new_func.__code__ = types.CodeType(nfcode.co_argcount - len(arg_name2value),
                                       nfcode.co_kwonlyargcount,
                                       len(new_varnames),
                                       nfcode.co_stacksize,
                                       nfcode.co_flags,
                                       new_co_code,
                                       new_consts,
                                       nfcode.co_names,
                                       new_varnames,
                                       nfcode.co_filename,
                                       name,
                                       nfcode.co_firstlineno,
                                       nfcode.co_lnotab,
                                       nfcode.co_freevars,
                                       nfcode.co_cellvars
                                      )                    
    
    return new_func

In [9]:
def toto(text, retext):
    print(text)
    a = "bonjour"
    b = 6
    print(a)
    return retext

In [30]:
new_toto = inlined_partial(toto, "new_toto", retext=5, text=2)

In [29]:
print_co(toto)

	co_argcount = 2
	co_cellvars = ()
	co_code = b't\x00|\x00\x83\x01\x01\x00d\x01}\x02d\x02}\x03t\x00|\x02\x83\x01\x01\x00|\x01S\x00'
	co_consts = (None, 'bonjour', 6)
	co_filename = <ipython-input-9-0488043ab200>
	co_firstlineno = 1
	co_flags = 67
	co_freevars = ()
	co_kwonlyargcount = 0
	co_lnotab = b'\x00\x01\x08\x01\x04\x01\x04\x01\x08\x01'
	co_name = toto
	co_names = ('print',)
	co_nlocals = 4
	co_stacksize = 2
	co_varnames = ('text', 'retext', 'a', 'b')


In [28]:
print_co(new_toto)

	co_argcount = 1
	co_cellvars = ()
	co_code = b't\x00|\x02\x83\x01\x01\x00d\x01}\x00d\x02}\x01t\x00|\x00\x83\x01\x01\x00d\x03S\x00'
	co_consts = (None, 'bonjour', 6, 5)
	co_filename = <ipython-input-9-0488043ab200>
	co_firstlineno = 1
	co_flags = 67
	co_freevars = ()
	co_kwonlyargcount = 0
	co_lnotab = b'\x00\x01\x08\x01\x04\x01\x04\x01\x08\x01'
	co_name = new_toto
	co_names = ('print',)
	co_nlocals = 3
	co_stacksize = 2
	co_varnames = ('a', 'b', 'text')


In [27]:
new_toto()

TypeError: new_toto() missing 1 required positional argument: 'a'

In [20]:
def titi(a, b):
    return a + b

In [ ]:
def inline(pre_func, func, pre_func_args=None):
    fcode = func.__code__
    
    inlined_partial_prefunc = inlined_partial(pre_func, "inlined_prefunc", **pre_func_args)
    ippcode = inlined_partial_prefunc.__code__
    
    
    new_func = copy_func(func, "new_func")
    new_consts = tuple_remove_duplicate(fcode.co_consts + ippcode.co_consts)

In [38]:
def outer_func(func):
    def inner_func(*args, **kwargs):
        x = 2
        y = math.sin(3)
        
        print(x + y)
        
        return func(*args, **kwargs)
    return inner_func

In [51]:
@inliner(outer_func)
def aa(a, b):
    c = 2 * a
    return c * b

In [40]:
bb = outer_func(aa)

In [47]:
aa(3, 4)

2.1411200080598674


24

In [42]:
bb(3, 4)

2.1411200080598674


24

In [52]:
print_co(aa)

	co_argcount = 0
	co_cellvars = ()
	co_code = b'd\x01}\x02t\x00\xa0\x01d\x02\xa1\x01}\x03t\x02|\x02|\x03\x17\x00\x83\x01\x01\x00\x88\x00|\x00|\x01\x8e\x01S\x00'
	co_consts = (None, 2, 3)
	co_filename = <ipython-input-38-e73876bf53cb>
	co_firstlineno = 2
	co_flags = 31
	co_freevars = ('func',)
	co_kwonlyargcount = 0
	co_lnotab = b'\x00\x01\x04\x01\n\x02\x0c\x02'
	co_name = inner_func
	co_names = ('math', 'sin', 'print')
	co_nlocals = 4
	co_stacksize = 3
	co_varnames = ('args', 'kwargs', 'x', 'y')


In [53]:
dis.dis(aa)

  3           0 LOAD_CONST               1 (2)
              2 STORE_FAST               2 (x)

  4           4 LOAD_GLOBAL              0 (math)
              6 LOAD_METHOD              1 (sin)
              8 LOAD_CONST               2 (3)
             10 CALL_METHOD              1
             12 STORE_FAST               3 (y)

  6          14 LOAD_GLOBAL              2 (print)
             16 LOAD_FAST                2 (x)
             18 LOAD_FAST                3 (y)
             20 BINARY_ADD
             22 CALL_FUNCTION            1
             24 POP_TOP

  8          26 LOAD_DEREF               0 (func)
             28 LOAD_FAST                0 (args)
             30 LOAD_FAST                1 (kwargs)
             32 CALL_FUNCTION_EX         1
             34 RETURN_VALUE


In [45]:
print_co(bb)

	co_argcount = 0
	co_cellvars = ()
	co_code = b'd\x01}\x02t\x00\xa0\x01d\x02\xa1\x01}\x03t\x02|\x02|\x03\x17\x00\x83\x01\x01\x00\x88\x00|\x00|\x01\x8e\x01S\x00'
	co_consts = (None, 2, 3)
	co_filename = <ipython-input-38-e73876bf53cb>
	co_firstlineno = 2
	co_flags = 31
	co_freevars = ('func',)
	co_kwonlyargcount = 0
	co_lnotab = b'\x00\x01\x04\x01\n\x02\x0c\x02'
	co_name = inner_func
	co_names = ('math', 'sin', 'print')
	co_nlocals = 4
	co_stacksize = 3
	co_varnames = ('args', 'kwargs', 'x', 'y')


In [57]:
def toto(x, y):
    print("bonjour")
    if x > y:
        return "sup"
    else:
        return "inf"

In [58]:
print_co(toto)

	co_argcount = 2
	co_cellvars = ()
	co_code = b't\x00d\x01\x83\x01\x01\x00|\x00|\x01k\x04r\x14d\x02S\x00d\x03S\x00d\x00S\x00'
	co_consts = (None, 'bonjour', 'sup', 'inf')
	co_filename = <ipython-input-57-4c40d8a2df96>
	co_firstlineno = 1
	co_flags = 67
	co_freevars = ()
	co_kwonlyargcount = 0
	co_lnotab = b'\x00\x01\x08\x01\x08\x01\x04\x02'
	co_name = toto
	co_names = ('print',)
	co_nlocals = 2
	co_stacksize = 2
	co_varnames = ('x', 'y')


In [59]:
dis.dis(toto)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_CONST               1 ('bonjour')
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_FAST                0 (x)
             10 LOAD_FAST                1 (y)
             12 COMPARE_OP               4 (>)
             14 POP_JUMP_IF_FALSE       20

  4          16 LOAD_CONST               2 ('sup')
             18 RETURN_VALUE

  6     >>   20 LOAD_CONST               3 ('inf')
             22 RETURN_VALUE
             24 LOAD_CONST               0 (None)
             26 RETURN_VALUE
